In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re
import html as html_lib

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/8.kit_part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：884



,[Kit] JOIN_MPNTCPC,[Kit] Type,[Kit] Manufacturer,[Kit] Part Number,[Kit] OEM,[Kit] Picture,[Kit] Url,[Kit] Json_Src,[Kit] Info,[Kit] Json_Note 1,[Kit] Json_Note 2,[Kit] Choose,[Kit] Json_Price,[Kit] Type Code,[Kit] Part Code
0,MELLING;1007;5724;8304536,Timing Chain,MELLING,1007,12566651;12586964;12590920;12597307,,"https://www.rockauto.com/en/parts/MELLING,1007...","{""0"": ""https://www.rockauto.com/info/583/1007-...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$31.79""}",5724,8304536
1,MELLING;1025;5724;8921164,Timing Chain,MELLING,1025,135060P010;135060P011;1350631020;1350631030;13...,,"https://www.rockauto.com/en/parts/MELLING,1025...","{""0"": ""https://www.rockauto.com/info/583/1025-...",https://www.rockauto.com/en/moreinfo.php?pk=89...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$59.79""}",5724,8921164
2,MELLING;1026;5724;8921168,Timing Chain,MELLING,1026,135070P010;1350731020,,"https://www.rockauto.com/en/parts/MELLING,1026...","{""0"": ""https://www.rockauto.com/info/583/1026-...",https://www.rockauto.com/en/moreinfo.php?pk=89...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$12.03""}",5724,8921168
3,MELLING;1035;5724;10454488,Timing Chain,MELLING,1035,14401RNAA01,,"https://www.rockauto.com/en/parts/MELLING,1035...","{""0"": ""https://www.rockauto.com/info/583/1035-...",https://www.rockauto.com/en/moreinfo.php?pk=10...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$43.89""}",5724,10454488
4,MELLING;1036;5724;8921172,Timing Chain,MELLING,1036,BR3Z6268B,,"https://www.rockauto.com/en/parts/MELLING,1036...","{""0"": ""https://www.rockauto.com/info/583/1036-...",https://www.rockauto.com/en/moreinfo.php?pk=89...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$35.79""}",5724,8921172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,MELLING;WP10022;2208;14645716,Water Pump,MELLING,WP10022,,,"https://www.rockauto.com/en/parts/MELLING,WP10...",{},https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Water Pump""}",,"{""0"": ""Out of Stock""}",2208,14645716
880,MELLING;WP10091;2208;14645720,Water Pump,MELLING,WP10091,,,"https://www.rockauto.com/en/parts/MELLING,WP10...",{},https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Water Pump""}",,"{""0"": ""Out of Stock""}",2208,14645720
881,MELLING;WP10510;2208;14645712,Water Pump,MELLING,WP10510,,,"https://www.rockauto.com/en/parts/MELLING,WP10...",{},https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Water Pump (Only 1 Remaining)""}",,"{""0"": ""$19.57""}",2208,14645712
882,MELLING;WP10516;2208;14645724,Water Pump,MELLING,WP10516,,,"https://www.rockauto.com/en/parts/MELLING,WP10...",{},https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Water Pump""}",,"{""0"": ""Out of Stock""}",2208,14645724


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 [Kit] JOIN_MPNTCPC                            MELLING;1007;5724;8304536
 [Kit] Type                                                 Timing Chain
 [Kit] Manufacturer                                              MELLING
 [Kit] Part Number                                                  1007
 [Kit] OEM                           12566651;12586964;12590920;12597307
 [Kit] Picture                                                          
 [Kit] Url             https://www.rockauto.com/en/parts/MELLING,1007...
 [Kit] Json_Src        {"0": "https://www.rockauto.com/info/583/1007-...
 [Kit] Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 [Kit] Json_Note 1                                                    {}
 [Kit] Json_Note 2                       {"0": "Category: Timing Chain"}
 [Kit] Choose                                                           
 [Kit] Json_Price                                        {"0": "$31.79"}
 [Kit] Type Code                       

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = crawler_series['[Kit] Info']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            if html.xpath('//span[@class="la-btn-v"]/text()')[0].strip() == 'Continue Shopping':
                with open('./kit_info.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

soup

1
2


<!DOCTYPE HTML>
<html class="no-js REPLACEHTMLCLASS" lang="en" onclick="">
<head>
<meta content="all" name="robots"/>
<meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
<link href="/jslib/combined/css/desktopmoreinfo.css?1729211056" rel="stylesheet" type="text/css"/>
<script>
if(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};
_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift();for(var d=windo

In [6]:
list_tr, dict_count = html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else [], {}
for i, tr in enumerate(list_tr):
    list_text = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
    list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
    list_text = [html_lib.unescape(text) for text in list_text]
    if len(list_text) > 2:
        print('fuck')
    elif list_text[0] in dict_count:
        dict_count[list_text[0]] += 1
    else:
        dict_count[list_text[0]] = 1

dict_specification, dict_no = {}, {}
for i, tr in enumerate(list_tr):
    dict_specification[str(i)] = {}

    list_text = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
    list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
    list_text = [html_lib.unescape(text) for text in list_text]
    if dict_count[list_text[0]] == 1:
        for j, text in enumerate(list_text):
            dict_specification[str(i)][str(j)] = text
    else:
        if list_text[0] in dict_no:
            dict_no[list_text[0]] += 1
        else:
            dict_no[list_text[0]] = 1
        for j, text in enumerate(list_text):
            if j == 0:
                dict_specification[str(i)][str(j)] = f'{text}-{dict_no[list_text[0]]}'
            else:
                dict_specification[str(i)][str(j)] = text

dict_specification

{'0': {'0': 'Chain Length-1', '1': '4.250"'},
 '1': {'0': 'Chain Length-2', '1': '107.95 MM'},
 '2': {'0': 'Chain Material', '1': 'Hardened Steel'},
 '3': {'0': 'Chain Type', '1': 'Silent Chain Flexible'},
 '4': {'0': 'Chain Width-1', '1': '0.450"'},
 '5': {'0': 'Chain Width-2', '1': '11.43 mm'},
 '6': {'0': 'Link Quantity', '1': '78'},
 '7': {'0': 'Row Quantity', '1': '1'},
 '8': {'0': 'Timing Mark Quantity', '1': '3'},
 '9': {'0': 'Timing Marks', '1': 'Yes'}}

In [ ]:
dict_kit, list_tr = {}, html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
for i, tr in enumerate(list_tr):
    dict_kit[str(i)] = {}

    list_td = tr.xpath('./td')
    dict_kit[str(i)]['[Kit] Quantity'] = int(list_td[0].xpath('./text()')[0].strip())

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['[Kit] Type'] = list_td[1].xpath('./text()')[0].strip()

    # = = = = = = = = = = = = = = =

    dict_ = json.loads(tr.xpath('./td[contains(@name, "listing_data_essential[")]/@value')[0])
    dict_kit[str(i)]['[Kit] Type Code'] = dict_['parttype']
    dict_kit[str(i)]['[Kit] Part Code'] = dict_['partkey']

    # = = = = = = = = = = = = = = =

    dict_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
    dict_kit[str(i)]['[Kit] Part Number'] = dict_2['partnumber']
    dict_kit[str(i)]['[Kit] Manufacturer'] = dict_2['catalogname']

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['[Kit] Url'] = f'''https://www.rockauto.com/en/parts/{dict_kit[str(i)]['[Kit] Manufacturer']},{dict_kit[str(i)]['[Kit] Part Number']},{dict_kit[str(i)]['[Kit] Type']},{dict_kit[str(i)]['[Kit] Type Code']}'''

    # = = = = = = = = = = = = = = =

    list_a = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
    info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
    dict_kit[str(i)]['[Kit] Info'] = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={dict_kit[str(i)]['[Kit] Part Code']}&pt={dict_kit[str(i)]['[Kit] Type Code']}&Lennon=1'''

    # = = = = = = = = = = = = = = =

    list_input = list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]')
    list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

    dict_src, j = {}, 0
    for dict_ in list_dict:
        list_src = dict_.get('Slots', [])
        for src in list_src:
            dict_src[str(j)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
            j += 1

    dict_kit[str(i)]['[Kit] Dict_Src'] = dict_src

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['[Kit] JOIN_MPNTCPC'] = f'''{dict_kit[str(i)]['[Kit] Manufacturer']};{dict_kit[str(i)]['[Kit] Part Number']};{dict_kit[str(i)]['[Kit] Type Code']};{dict_kit[str(i)]['[Kit] Part Code']}'''

dict_kit

{}

In [8]:
list_dict_correct.append({'[Kit] JOIN_MPNTCPC': crawler_series['[Kit] JOIN_MPNTCPC'],
                          '[Kit] Type': crawler_series['[Kit] Type'],
                          '[Kit] Manufacturer': crawler_series['[Kit] Manufacturer'],
                          '[Kit] Part Number': crawler_series['[Kit] Part Number'],
                          '[Kit] OEM': crawler_series['[Kit] OEM'],
                          '[Kit] Picture': crawler_series['[Kit] Picture'],
                          '[Kit] Url': crawler_series['[Kit] Url'],
                          '[Kit] Json_Src': crawler_series['[Kit] Json_Src'],
                          '[Kit] Info': crawler_series['[Kit] Info'],
                          '[Kit] Json_Note 1': crawler_series['[Kit] Json_Note 1'],
                          '[Kit] Json_Note 2': crawler_series['[Kit] Json_Note 2'],
                          '[Kit] Choose': crawler_series['[Kit] Choose'],
                          '[Kit] Json_Price': crawler_series['[Kit] Json_Price'],
                          '[Kit] Json_Specification': json.dumps(dict_specification),
                          '[Kit] Json_Kit': json.dumps(dict_kit),
                          '[Kit] Type Code': crawler_series['[Kit] Type Code'],
                          '[Kit] Part Code': crawler_series['[Kit] Part Code']})

list_dict_correct

[{'[Kit] JOIN_MPNTCPC': 'MELLING;1007;5724;8304536',
  '[Kit] Type': 'Timing Chain',
  '[Kit] Manufacturer': 'MELLING',
  '[Kit] Part Number': '1007',
  '[Kit] OEM': '12566651;12586964;12590920;12597307',
  '[Kit] Picture': '',
  '[Kit] Url': 'https://www.rockauto.com/en/parts/MELLING,1007,Timing Chain,5724',
  '[Kit] Json_Src': '{"0": "https://www.rockauto.com/info/583/1007-1-ANG.jpg"}',
  '[Kit] Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8304536&cc=0&pt=5724',
  '[Kit] Json_Note 1': '{}',
  '[Kit] Json_Note 2': '{"0": "Category: Timing Chain"}',
  '[Kit] Choose': '',
  '[Kit] Json_Price': '{"0": "$31.79"}',
  '[Kit] Json_Specification': '{"0": {"0": "Chain Length-1", "1": "4.250\\""}, "1": {"0": "Chain Length-2", "1": "107.95 MM"}, "2": {"0": "Chain Material", "1": "Hardened Steel"}, "3": {"0": "Chain Type", "1": "Silent Chain Flexible"}, "4": {"0": "Chain Width-1", "1": "0.450\\""}, "5": {"0": "Chain Width-2", "1": "11.43 mm"}, "6": {"0": "Link Quantity", "1": "78"}, "7": {

In [9]:
crawler_status = True

crawler_status

True

In [10]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[[Kit] JOIN_MPNTCPC                            MELLING;1007;5724;8304536
 [Kit] Type                                                 Timing Chain
 [Kit] Manufacturer                                              MELLING
 [Kit] Part Number                                                  1007
 [Kit] OEM                           12566651;12586964;12590920;12597307
 [Kit] Picture                                                          
 [Kit] Url             https://www.rockauto.com/en/parts/MELLING,1007...
 [Kit] Json_Src        {"0": "https://www.rockauto.com/info/583/1007-...
 [Kit] Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 [Kit] Json_Note 1                                                    {}
 [Kit] Json_Note 2                       {"0": "Category: Timing Chain"}
 [Kit] Choose                                                           
 [Kit] Json_Price                                        {"0": "$31.79"}
 [Kit] Type Code                                   

In [11]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['[Kit] JOIN_MPNTCPC'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_kit_info.xlsx', index=False)

df_correct

,[Kit] JOIN_MPNTCPC,[Kit] Type,[Kit] Manufacturer,[Kit] Part Number,[Kit] OEM,[Kit] Picture,[Kit] Url,[Kit] Json_Src,[Kit] Info,[Kit] Json_Note 1,[Kit] Json_Note 2,[Kit] Choose,[Kit] Json_Price,[Kit] Json_Specification,[Kit] Json_Kit,[Kit] Type Code,[Kit] Part Code
0,MELLING;1007;5724;8304536,Timing Chain,MELLING,1007,12566651;12586964;12590920;12597307,,"https://www.rockauto.com/en/parts/MELLING,1007...","{""0"": ""https://www.rockauto.com/info/583/1007-...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$31.79""}","{""0"": {""0"": ""Chain Length-1"", ""1"": ""4.250\""""},...",{},5724,8304536


In [12]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['[Kit] JOIN_MPNTCPC'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_kit_info_error.xlsx', index=False)

df_error

,[Kit] JOIN_MPNTCPC,[Kit] Type,[Kit] Manufacturer,[Kit] Part Number,[Kit] OEM,[Kit] Picture,[Kit] Url,[Kit] Json_Src,[Kit] Info,[Kit] Json_Note 1,[Kit] Json_Note 2,[Kit] Choose,[Kit] Json_Price,[Kit] Type Code,[Kit] Part Code,Request_Url
0,MELLING;1007;5724;8304536,Timing Chain,MELLING,1007,12566651;12586964;12590920;12597307,,"https://www.rockauto.com/en/parts/MELLING,1007...","{""0"": ""https://www.rockauto.com/info/583/1007-...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain""}",,"{""0"": ""$31.79""}",5724,8304536,https://www.rockauto.com/en/moreinfo.php?pk=83...
